In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827 cell culture'


In [5]:
from collections import defaultdict

data_dir = (Path().cwd().parents[0] / 'data').absolute()

# Get image dataframe 
df_imgs_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_imgs_path)


# Read per cell data

In [6]:
# Read PPI dfs
PPI_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
df_subset = df[(df.Cell > 0) & (df.Nuclei == 0)]
g = df_subset .groupby(['Condition', 'FOV', 'PPI', 'Cell']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

df_subset = df[(df.Cell > 0) & (df.Nuclei > 0)]
g = df_subset .groupby(['Condition', 'FOV', 'PPI', 'Cell']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,9810.000000,9810.000000,9810.00000,9810.000000
mean,698.097044,22.324465,9.90316,12.421305
std,518.136602,34.612066,11.76159,27.293782
min,2.000000,1.000000,0.00000,-47.000000
25%,253.000000,3.000000,1.00000,1.000000
50%,529.000000,11.000000,6.00000,4.000000
75%,1184.000000,24.000000,14.00000,13.000000
max,1726.000000,787.000000,165.00000,787.000000


,Cell,Nuclei,Cyto
count,1455.000000,1455.000000,1455.000000
mean,150.517526,66.769759,83.747766
std,98.439213,37.212778,85.072663
min,1.000000,0.000000,-63.000000
25%,88.000000,44.000000,33.000000
50%,132.000000,61.000000,66.000000
75%,188.000000,83.000000,114.500000
max,1298.000000,357.000000,1298.000000


In [7]:
# Filter out by maximum number of counts per cell
min_count = 80
max_count = 400

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,7962.000000,7962.000000,7962.000000,7962.000000
mean,719.072092,24.677342,10.525371,14.151972
std,527.023985,35.008496,12.306788,26.842991
min,4.000000,1.000000,0.000000,-47.000000
25%,254.000000,4.000000,1.000000,1.000000
50%,540.000000,13.000000,7.000000,5.000000
75%,1243.000000,27.000000,15.000000,15.000000
max,1726.000000,384.000000,165.000000,359.000000


,Cell,Nuclei,Cyto
count,1141.000000,1141.000000,1141.000000
mean,172.200701,73.446976,98.753725
std,82.949620,36.277996,73.403402
min,81.000000,0.000000,-63.000000
25%,114.000000,51.000000,52.000000
50%,152.000000,68.000000,84.000000
75%,206.000000,89.000000,127.000000
max,915.000000,357.000000,810.000000


# ML model

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [9]:
condition = 'ML'
project_name = f'PLA_9PPI_070323_3D_{condition}'

In [10]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [11]:
df_count

Cell                                               \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin E & CDK4 Mcl-1 & BAK   
Condition  FOV Id                                                             
HCC827Ctrl FW1 16         12.0             90.0             0.0         0.0   
               17         24.0            261.0             0.0         2.0   
               18         26.0            179.0             2.0         1.0   
               21         15.0            104.0             0.0         0.0   
               27          7.0             70.0             0.0         1.0   
...                        ...              ...             ...         ...   
HCC827Osim FW2 528        20.0             40.0             0.0         1.0   
               529        34.0             73.0             2.0         0.0   
               531        28.0            127.0             0.0         0.0   
               534        18.0             81.0             1.0         2.0   
               535        41.0             76.0             2.0         0.0   

                                                                          \
PPI                NF-Kb & p-P90rsk P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                          
HCC827Ctrl FW1 16              28.0          1.0        28.0         1.0   
               17              45.0          0.0        59.0         2.0   
               18              42.0          1.0        47.0         2.0   
               21              20.0          0.0        19.0         0.0   
               27              11.0          0.0        10.0         0.0   
...                             ...          ...         ...         ...   
HCC827Osim FW2 528             14.0          1.0         9.0         3.0   
               529             25.0          3.0        30.0         2.0   
               531             32.0          0.0        50.0         2.0   
               534             20.0          0.0        28.0         1.0   
               535             31.0          0.0        29.0         3.0   

                                      Nuclei  ...                      Cyto  \
PPI                p-ERK & c-MYC Bim & Tom20  ... p-ERK & c-MYC Bim & Tom20   
Condition  FOV Id                             ...                             
HCC827Ctrl FW1 16           20.0        26.0  ...           9.0       -14.0   
               17            7.0        35.0  ...           9.0       -11.0   
               18           23.0        26.0  ...           8.0         0.0   
               21            4.0        19.0  ...           3.0        -4.0   
               27            2.0         3.0  ...           1.0         4.0   
...                          ...         ...  ...           ...         ...   
HCC827Osim FW2 528           7.0         7.0  ...           0.0        13.0   
               529          12.0        19.0  ...           1.0        15.0   
               531           9.0        16.0  ...           4.0        12.0   
               534          15.0        30.0  ...          11.0       -12.0   
               535           9.0        26.0  ...          13.0        15.0   

                                                                 \
PPI                Cyclin D1 & CDK2 Cyclin E & CDK4 Mcl-1 & BAK   
Condition  FOV Id                                                 
HCC827Ctrl FW1 16              46.0             0.0         0.0   
               17             201.0             0.0         1.0   
               18             139.0             2.0         0.0   
               21              83.0             0.0         0.0   
               27              51.0             0.0         1.0   
...                             ...             ...         ...   
HCC827Osim FW2 528             14.0             0.0        -1.0   
               529             61.0             2.0         0.0   
          

In [12]:
df_count.reset_index().groupby('Condition').size()

Condition
HCC827Ctrl    699
HCC827Osim    442
dtype: int64

In [38]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True), 
}

In [39]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.67249
auc,0.65795
f1,0.59016


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.73684
auc,0.69562
f1,0.6


...

accuracy,▁
auc,▁
f1,▁
accuracy,0.73246
auc,0.67042
f1,0.54815


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.68421
auc,0.62496
f1,0.47826


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.66228
auc,0.60154
f1,0.384


In [40]:
# Run model on cell count
X = df_count_cyto_nuclei
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cyto_nuclei', name=model_name+f'_cyto_nuclei_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cyto_nuclei', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.67249
auc,0.65064
f1,0.55621


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.72807
auc,0.69679
f1,0.60759


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
f1,▁
accuracy,0.77632
auc,0.72182
f1,0.62774


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.71053
auc,0.66214
f1,0.54795


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.70175
auc,0.66211
f1,0.54667
